<center><img src='https://claudemonetgallery.org/thumbnail/81000/81127/mini_small/Self-Portrait-With-A-Beret.jpg?ts=1459229076' height=350></center>
<p>
<h1><center> I’m Something of a Painter Myself </center></h1>
<h2><center> Introduction to CycleGAN - Monet paintings </center></h2>

#### This notebook is based on the [competition baseline](https://www.kaggle.com/amyjang/monet-cyclegan-tutorial), I just did some refactoring to create helper functions and make everything easier to experiment with, besides that the main contribution is adding the possibility to use data augmentations, as we have very little data here, this will probably help.

#### CycleGAN references:
- [Git repository](https://junyanz.github.io/CycleGAN/) with many cool informations.
- [ArXiv paper](https://arxiv.org/pdf/1703.10593.pdf)
- [Understanding and Implementing CycleGAN in TensorFlow](https://hardikbansal.github.io/CycleGANBlog/)


### What is CycleGAN?

From the authors:
> We present an approach for learning to translate an image from a source domain X to a target domain Y in the absence of paired examples. Our goal is to learn a mapping G: X → Y, such that the distribution of images from G(X) is indistinguishable from the distribution Y using an adversarial loss. Because this mapping is highly under-constrained, we couple it with an inverse mapping F: Y → X and introduce a cycle consistency loss to push F(G(X)) ≈ X (and vice versa).

In essence it maps and image to a given domaind, if you are turning horses into zebra the image will be the horse and the domain is the zebras, in our case the photos are the image and the domain are the Monet paintings.

#### Turning horses into zebras and zebras into horses
![](https://raw.githubusercontent.com/dimitreOliveira/MachineLearning/master/Kaggle/I%E2%80%99m%20Something%20of%20a%20Painter%20Myself/cyclegan_horse-zebra.jpg)

#### Turning photos into Monet paintings (our task)
![](https://junyanz.github.io/CycleGAN/images/painting2photo.jpg)


But it doesn't always works as expected
<img src='https://junyanz.github.io/CycleGAN/images/failure_putin.jpg' height=300, width=300>

### CycleGAN architecture

Looking at the code below may be hard to get what is happening, this image will help the understanding

<img src='https://hardikbansal.github.io/CycleGANBlog/images/model.jpg' height=700, width=700>

First, we get the regular generator discriminator thing, where the generator tries to generate images that seem to be drawn to the given domain (in the example will be creating zebra images), but it would be possible that the generator generates only the same zebra image or zebra images that do not look like the imputed horse image, this is why the model has a second generator, this second generator uses the first generated image and tries to recreate the original imputed horse image, this way the first generator has to generate zebra images that look like the imputed horse image.

#### In the end, you will get 4 sub-models:
- A generator that can generate zebras images
- A generator that can generate horses images
- A discriminator that can identify real zebras images
- A discriminator that can identify real horses images


#### Let's get to the code

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.callbacks import History
import matplotlib.pyplot as plt
import numpy as np
import re
import os
import math
import random
import cv2

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()

In [ ]:
GCS_PATH = KaggleDatasets().get_gcs_path('gan-getting-started')
fn_monet = tf.io.gfile.glob(str(GCS_PATH + '/monet_jpg/*.jpg'))
fn_photo = tf.io.gfile.glob(str(GCS_PATH + '/photo_jpg/*.jpg'))

# **Display Images**

In [ ]:
# View one image
import imageio
photo_image_names = os.listdir('../input/gan-getting-started/photo_jpg')
photo_img = imageio.imread(os.path.join('../input/gan-getting-started/photo_jpg', photo_image_names[105]))
plt.imshow(photo_img)

plt.figure()

In [ ]:
# View one image
import imageio
monet_image_names = os.listdir('../input/gan-getting-started/monet_jpg')
monet_img = imageio.imread(os.path.join('../input/gan-getting-started/monet_jpg', monet_image_names[105]))
plt.imshow(monet_img)

plt.figure()

# **Image preprocessing**

In [ ]:
rand_monet = r"../input/gan-getting-started/monet_jpg/0260d15306.jpg"
rand_photo = r"../input/gan-getting-started/photo_jpg/000ded5c41.jpg"

In [ ]:
def color_graph(image_path, figsize=(16, 4)):
    plt.figure(figsize=figsize)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.axis('off')
    
    chans = cv2.split(img)
    colors = ("b", "g", "r")
    plt.subplot(1, 2, 2)
    plt.title("'Flattened' Color Histogram")
    plt.xlabel("Bins")
    plt.ylabel("# of Pixels")
    features = []
    # loop over the image channels
    for (chan, color) in zip(chans, colors):
        # create a histogram for the current channel and
        # concatenate the resulting histograms for each
        # channel
        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        features.extend(hist)
        # plot the histogram
        plt.plot(hist, color = color)
        plt.xlim([0, 256])
    
    plt.show()

In [ ]:
print("Monet: ")
color_graph(rand_monet)
print("\nPhoto: ")
color_graph(rand_photo)

In [ ]:
BATCH_SIZE =  4


def parse_function(filename):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = (tf.cast(image,tf.float32)/ 127.5) - 1
    image = tf.reshape(image, [256, 256,3])
    return image

def data_augment(image):
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    if p_crop > .5:
        image = tf.image.resize(image, [286, 286])
        image = tf.image.random_crop(image, size=[256, 256, 3])
        if p_crop > .9:
            image = tf.image.resize(image, [300, 300])
            image = tf.image.random_crop(image, size=[256, 256, 3])
    
    if p_rotate > .9:
        image = tf.image.rot90(image, k=3)
    elif p_rotate > .7:
        image = tf.image.rot90(image, k=2)
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=1)
        
    if p_spatial > .6:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        if p_spatial > .9:
            image = tf.image.transpose(image)
    
    return image

num_parallel_calls=tf.data.experimental.AUTOTUNE
def getSet(filenames):
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    dataset = dataset.shuffle(len(filenames))
    dataset = dataset.map(parse_function, num_parallel_calls)
    dataset = dataset.map(data_augment, num_parallel_calls)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
    dataset = dataset.cache()
    dataset = dataset.prefetch(num_parallel_calls)
    return dataset

monet_ds=getSet(fn_monet)
photo_ds=getSet(fn_photo)

# **Model Building**

We'll be using a UNET architecture for our CycleGAN. To build our generator, let's first define our downsample and upsample methods.

The downsample, as the name suggests, reduces the 2D dimensions, the width and height, of the image by the stride. The stride is the length of the step the filter takes. Since the stride is 2, the filter is applied to every other pixel, hence reducing the weight and height by 2.

We'll be using an instance normalization instead of batch normalization. As the instance normalization is not standard in the TensorFlow API, we'll use the layer from TensorFlow Add-ons.

In [ ]:
OUTPUT_CHANNELS = 3

def downsample(filters, size, apply_instancenorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    result = keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same',kernel_initializer=initializer, use_bias=False))

    if apply_instancenorm:
        result.add(tfa.layers.InstanceNormalization(gamma_initializer=gamma_init))

    result.add(layers.LeakyReLU())

    return result

Upsample does the opposite of downsample and increases the dimensions of the of the image. Conv2DTranspose does basically the opposite of a Conv2D layer.

In [ ]:
def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    result = keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2,padding='same',kernel_initializer=initializer,use_bias=False))

    result.add(tfa.layers.InstanceNormalization(gamma_initializer=gamma_init))

    if apply_dropout:
        result.add(layers.Dropout(0.5))

    result.add(layers.ReLU())

    return result

# **Generator**

The generator first downsamples the input image and then upsample while establishing long skip connections. Skip connections are a way to help bypass the vanishing gradient problem by concatenating the output of a layer to multiple layers instead of only one. Here we concatenate the output of the downsample layer to the upsample layer in a symmetrical fashion.
64-->512=Encoder, 512-->Till Upsample droput 512,4 with drop out is -->Transformer and 512-->64 is decoder

In [ ]:
def Generator():
    inputs = layers.Input(shape=[256,256,3])

    # bs = batch size
    down_stack = [
        downsample(64, 4, apply_instancenorm=False), # (bs, 128, 128, 64) 256/2（padding same，(256-4+1)/2）
        downsample(128, 4), # (bs, 64, 64, 128)
        downsample(256, 4), # (bs, 32, 32, 256)
        downsample(512, 4), # (bs, 16, 16, 512)
        downsample(512, 4), # (bs, 8, 8, 512)
        downsample(512, 4), # (bs, 4, 4, 512)
        downsample(512, 4), # (bs, 2, 2, 512)
        downsample(512, 4), # (bs, 1, 1, 512)
    ]

    up_stack = [
        upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024) 1*2（padding same，(1*2-4+1)），，
        upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
        upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
        upsample(512, 4), # (bs, 16, 16, 1024)
        upsample(256, 4), # (bs, 32, 32, 512)
        upsample(128, 4), # (bs, 64, 64, 256)
        upsample(64, 4), # (bs, 128, 128, 128)
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,strides=2,padding='same',kernel_initializer=initializer,activation='tanh') # (bs, 256, 256, 3)，，

    x = inputs

  
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])

    x = last(x)

    return keras.Model(inputs=inputs, outputs=x)

In [ ]:
tf.keras.utils.plot_model(Generator(), show_shapes=True, dpi=64)

# **Discriminator**

The discriminator takes in the input image and classifies it as real or fake (generated). Instead of outputing a single node, the discriminator outputs a smaller 2D image with higher pixel values indicating a real classification and lower values indicating a fake classification.
Each layer of discriminator has a instance normal lization so as to normalize the model and last has the choice between real and fake done by sigmoid.

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    inp = layers.Input(shape=[256, 256, 3], name='input_image')

    x = inp

    down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
    down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
    down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

    zero_pad1 = layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
    conv = layers.Conv2D(512, 4, strides=1,kernel_initializer=initializer,use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

    norm1 = tfa.layers.InstanceNormalization(gamma_initializer=gamma_init)(conv)

    leaky_relu = layers.LeakyReLU()(norm1)

    zero_pad2 = layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

    last = layers.Conv2D(1, 4, strides=1,
                         kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)，，

    return tf.keras.Model(inputs=inp, outputs=last)

In [ ]:
tf.keras.utils.plot_model(Discriminator(), show_shapes=True, dpi=64)

# **CycleGan**

**Training a CycleGAN**:
In the CycleGAN’s case, the architecture is complex, and as a result, we need a structure that allows us to keep accessing the original attributes and methods that we have defined. As a result, we will write out the CycleGAN as a Python class of its own with methods to build the Generator and Discriminator, and run the training.

For the training to execute we will need a seperate Generator() and discriminator() function which we will feed to CycleGAN as methods which in turn needs the upsample() and downsample() of image.

For downsampling we are using the Conv2D() as primary layer and LeakyReLU() as activation
For upsampling we are using the Conv2DTranspose() as primary layer and Dropout() at 0.3, ReLU() as secondary layers

In [ ]:
class CycleGan(keras.Model):
    def __init__(self,monet_generator,photo_generator,monet_discriminator,photo_discriminator,lambda_cycle=15):
        super(CycleGan, self).__init__()
        self.m_gen = monet_generator
        self.p_gen = photo_generator
        self.m_disc = monet_discriminator
        self.p_disc = photo_discriminator
        self.lambda_cycle = lambda_cycle
        
    def compile(self,m_gen_optimizer,p_gen_optimizer,m_disc_optimizer,p_disc_optimizer,gen_loss_fn,disc_loss_fn,cycle_loss_fn,identity_loss_fn):
        super(CycleGan, self).compile()
        self.m_gen_optimizer = m_gen_optimizer
        self.p_gen_optimizer = p_gen_optimizer
        self.m_disc_optimizer = m_disc_optimizer
        self.p_disc_optimizer = p_disc_optimizer
        
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn
        self.cycle_loss_fn = cycle_loss_fn
        self.identity_loss_fn = identity_loss_fn
        
    def train_step(self, batch_data):
        real_monet, real_photo = batch_data
        #real_monet y，real_photo x，m_gen G，p_gen F，p_disc DX，m_disc DY
        
        with tf.GradientTape(persistent=True) as tape:
            fake_monet = self.m_gen(real_photo, training=True)#G(x)
            cycled_photo = self.p_gen(fake_monet, training=True)#F(G(x))

            fake_photo = self.p_gen(real_monet, training=True)#F(y)
            cycled_monet = self.m_gen(fake_photo, training=True)#G(F(y))

            same_monet = self.m_gen(real_monet, training=True)#G(y)
            same_photo = self.p_gen(real_photo, training=True)#F(x)

            disc_real_monet = self.m_disc(real_monet, training=True)#DY(y)
            disc_real_photo = self.p_disc(real_photo, training=True)#DX(x)

            disc_fake_monet = self.m_disc(fake_monet, training=True)#DY(G(x))
            disc_fake_photo = self.p_disc(fake_photo, training=True)#DX(F(y))

            monet_gen_loss = self.gen_loss_fn(disc_fake_monet)
            photo_gen_loss = self.gen_loss_fn(disc_fake_photo)

            total_cycle_loss = self.cycle_loss_fn(real_monet, cycled_monet, self.lambda_cycle) + self.cycle_loss_fn(real_photo, cycled_photo, self.lambda_cycle)

            # evaluates total generator loss
            total_monet_gen_loss = monet_gen_loss + total_cycle_loss + self.identity_loss_fn(real_monet, same_monet, self.lambda_cycle)
            total_photo_gen_loss = photo_gen_loss + total_cycle_loss + self.identity_loss_fn(real_photo, same_photo, self.lambda_cycle)

            # evaluates discriminator loss
            monet_disc_loss = self.disc_loss_fn(disc_real_monet, disc_fake_monet)
            photo_disc_loss = self.disc_loss_fn(disc_real_photo, disc_fake_photo)

       
        monet_generator_gradients = tape.gradient(total_monet_gen_loss,self.m_gen.trainable_variables)
        photo_generator_gradients = tape.gradient(total_photo_gen_loss,self.p_gen.trainable_variables)
        monet_discriminator_gradients = tape.gradient(monet_disc_loss,self.m_disc.trainable_variables)
        photo_discriminator_gradients = tape.gradient(photo_disc_loss,self.p_disc.trainable_variables)

        self.m_gen_optimizer.apply_gradients(zip(monet_generator_gradients,self.m_gen.trainable_variables))
        self.p_gen_optimizer.apply_gradients(zip(photo_generator_gradients,self.p_gen.trainable_variables))
        self.m_disc_optimizer.apply_gradients(zip(monet_discriminator_gradients,self.m_disc.trainable_variables))
        self.p_disc_optimizer.apply_gradients(zip(photo_discriminator_gradients,self.p_disc.trainable_variables))
        
        '''if (iteration + 1) % sample_interval == 0:

            # Save losses and accuracies so they can be plotted after training
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)

            # Output training progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (iteration + 1, d_loss, 100.0 * accuracy, g_loss))

            # Output a sample of generated image
            sample_images(generator)'''
        #https://github.com/GANs-in-Action/gans-in-action/blob/master/chapter-3/Chapter_3_GAN.ipynb
        
        return {"monet_gen_loss": total_monet_gen_loss,"photo_gen_loss": total_photo_gen_loss,"monet_disc_loss": monet_disc_loss,"photo_disc_loss": photo_disc_loss}

# Discrimiator's loss

The discriminator loss function below compares real images to a matrix of 1s and fake images to a matrix of 0s. The perfect discriminator will output all 1s for real images and all 0s for fake images. The discriminator loss outputs the average of the real and generated loss.

In [ ]:
with strategy.scope():
    def discriminator_loss(real, generated):
        real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.ones_like(real), real)#对于真实样本，通过鉴别器后与全1矩阵计算交叉熵，使得D对真实数据输出尽可能接近1
        generated_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.zeros_like(generated), generated)#对于生成样本，通过鉴别器后与全0矩阵计算交叉熵，使得D对生成数据输出尽可能接近0
        total_disc_loss = real_loss + generated_loss
        return total_disc_loss * 0.5

The generator wants to fool the discriminator into thinking the generated image is real. The perfect generator will have the discriminator output only 1s. Thus, it compares the generated image to a matrix of 1s to find the loss.

In [ ]:
with strategy.scope():
    def generator_loss(generated):
        return tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.ones_like(generated), generated)

We want our original photo and the twice transformed photo to be similar to one another. Thus, we can calculate the cycle consistency loss be finding the average of their difference.

In [ ]:
with strategy.scope():
    def calc_cycle_loss(real_image, cycled_image, LAMBDA):
        loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
        return LAMBDA * loss1

The identity loss compares the image with its generator (i.e. photo with photo generator). If given a photo as input, we want it to generate the same image as the image was originally a photo. The identity loss compares the input with the output of the generator.

In [ ]:
with strategy.scope():
    def identity_loss(real_image, same_image, LAMBDA):
        loss = tf.reduce_mean(tf.abs(real_image - same_image))
        return LAMBDA * 0.5 * loss

In [ ]:
with strategy.scope():
    monet_generator_optimizer = tf.keras.optimizers.Adamax(2e-4, beta_1=0.5)
    photo_generator_optimizer = tf.keras.optimizers.Adamax(2e-4, beta_1=0.5)
    monet_discriminator_optimizer = tf.keras.optimizers.Adamax(2e-4, beta_1=0.5)
    photo_discriminator_optimizer = tf.keras.optimizers.Adamax(2e-4, beta_1=0.5)

In [ ]:
global im_to_gif
im_to_gif = np.zeros((30,256,256,3))
photo = next(iter(monet_ds))
num_photo = 0
plt.imshow(photo[num_photo]*0.5 + 0.5)

In [ ]:
class GANMonitor(keras.callbacks.Callback):
   """A callback to generate and save images after each epoch"""

   def on_epoch_end(self, epoch, logs=None):
        prediction =  monet_generator(photo, training=False)[num_photo].numpy()
        prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
        im_to_gif[epoch] = prediction   

In [ ]:
with strategy.scope():
    monet_generator=Generator()
    photo_generator=Generator()
    monet_discriminator=Discriminator()
    photo_discriminator=Discriminator()
with strategy.scope():
    cycle_gan_model = CycleGan(monet_generator,photo_generator,monet_discriminator,photo_discriminator)
    cycle_gan_model.compile(
            m_gen_optimizer = monet_generator_optimizer,
            p_gen_optimizer = photo_generator_optimizer,
            m_disc_optimizer = monet_discriminator_optimizer,
            p_disc_optimizer = photo_discriminator_optimizer,
            gen_loss_fn = generator_loss,
            disc_loss_fn = discriminator_loss,
            cycle_loss_fn = calc_cycle_loss,
            identity_loss_fn = identity_loss
        )
plotter = GANMonitor()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='generator_loss', mode='min', patience=1,restore_best_weights=True)

In [ ]:
steps_per_epoch=(max(len(fn_monet), len(fn_photo)))//BATCH_SIZE
history=cycle_gan_model.fit(tf.data.Dataset.zip((monet_ds, photo_ds)),epochs=25,steps_per_epoch=steps_per_epoch,callbacks=[History(),plotter])

In [ ]:
monet_generator.save('gen_monet.h5')
photo_generator.save('gen_photo.h5')
monet_discriminator.save('disc_monet.h5')
photo_discriminator.save('disc_photo.h5')
import pickle

In [ ]:
with open('history.pkl','wb') as f:
    pickle.dump(history.history, f)

In [ ]:
gen_monet = tf.keras.models.load_model('./gen_monet.h5')
gen_photo = tf.keras.models.load_model('./gen_photo.h5')
disc_monet = tf.keras.models.load_model('./disc_monet.h5')
disc_photo = tf.keras.models.load_model('./disc_photo.h5')

_*Got the idea from https://www.kaggle.com/matkneky/monet-cyclegan-trials*_

In [ ]:
def plot_acc_and_loss(history, load=False):
    monet_g = []
    photo_g = []
    monet_d = []
    photo_d = []
    if load==True:
        for i in range(np.array(history["monet_gen_loss"]).shape[0]):
            monet_g.append(np.array(history["monet_gen_loss"][i]).squeeze().mean())
            photo_g.append(np.array(history["photo_gen_loss"][i]).squeeze().mean())
            monet_d.append(np.array(history["monet_disc_loss"][i]).squeeze().mean())
            photo_d.append(np.array(history["photo_disc_loss"][i]).squeeze().mean())
    else:
        for i in range(np.array(history.history["monet_gen_loss"]).shape[0]):
            monet_g.append(np.array(history.history["monet_gen_loss"][i]).squeeze().mean())
            photo_g.append(np.array(history.history["photo_gen_loss"][i]).squeeze().mean())
            monet_d.append(np.array(history.history["monet_disc_loss"][i]).squeeze().mean())
            photo_d.append(np.array(history.history["photo_disc_loss"][i]).squeeze().mean())
    
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(monet_g,label="Monet")
    axs[0].plot(photo_g,label="Photo")
    axs[0].set_title("generator loss")
    axs[0].legend()

    axs[1].plot(monet_d, label="Monet")
    axs[1].plot(photo_d,label="Photo")
    axs[1].set_title("discriminator loss")
    axs[1].legend()

    plt.show()

In [ ]:
# Call this if loading outputs
import pickle
with (open("./history.pkl", "rb")) as openfile:
    history = pickle.load(openfile)

In [ ]:
history.keys()

In [ ]:
# Switch 'load' to false if you have trained the model
plot_acc_and_loss(history, load=True)

In [ ]:
def create_gif(num_photo=0, load=False):
    if load == True:
        anim_file = '../input/cyclegangif/CycleGAN.gif'
    else:
        # Creating a gif from each predictions
        anim_file = 'CycleGAN.gif'
        init_pic = np.array(photo[num_photo]*0.5 + 0.5)
        with imageio.get_writer(anim_file, mode='I') as writer:
            # Three first frames are the converted picture
            writer.append_data(init_pic)
            writer.append_data(init_pic)
            writer.append_data(init_pic)
            for i in range(im_to_gif.shape[0]):
                writer.append_data(im_to_gif[i])
                writer.append_data(im_to_gif[i])
                writer.append_data(im_to_gif[i])
            for i in range(int(im_to_gif.shape[0])):
                writer.append_data(im_to_gif[-1])
    return anim_file

In [ ]:
# Switch 'load' to false if you have trained the model and put the correct photo number
anim_file = create_gif(num_photo=num_photo,
                       load=True)

In [ ]:
def gen_input_img(num_photo=0, load=False):
    fig, ax = plt.subplots(figsize=(5,5))
    
    if load == True:
        img = np.array(PIL.Image.open('../input/gan-getting-started/monet_jpg/000c1e3bff.jpg'))
        plt.imshow(img)
        ax.axis("off")
        
    else:
        img = photo[3]*0.5 + 0.5
        plt.imshow(img)
        ax.axis("off")
        plt.title('Input photo')    


In [ ]:
# Switch 'load' to false if you have trained the model and put the correct photo number
gen_input_img(num_photo=num_photo,
              load=False)

In [ ]:
import PIL
from IPython import display
import imageio

import shutil

In [ ]:
# Switch 'load' to false if you have trained the model and put the correct photo number
gen_input_img(num_photo=num_photo,
              load=True)

In [ ]:
! pip install git+https://github.com/tensorflow/docs

In [ ]:
import tensorflow_docs.vis.embed as embed

In [ ]:
# Prediction evolution according to epoch
embed.embed_file(anim_file)

In [ ]:
# Switch 'load' to false if you have trained the model and put the correct photo number
anim_file = create_gif(num_photo=num_photo,
                       load=False)

In [ ]:
# Prediction evolution according to epoch
embed.embed_file(anim_file)

In [ ]:
testset = tf.data.Dataset.from_tensor_slices(fn_photo)
testset = testset.shuffle(len(fn_photo))
testset = testset.map(parse_function, num_parallel_calls)
testset=testset.batch(1)
testset = testset.prefetch(num_parallel_calls)


_, ax = plt.subplots(5, 2, figsize=(20, 20))
for i, img in enumerate(testset.take(5)):
    prediction = monet_generator(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

    ax[i, 0].imshow(img)
    ax[i, 1].imshow(prediction)
    ax[i, 0].set_title("Input Photo")
    ax[i, 1].set_title("Monet-esque")
    ax[i, 0].axis("off")
    ax[i, 1].axis("off")
plt.show()

In [ ]:
testset = tf.data.Dataset.from_tensor_slices(fn_monet)
testset = testset.shuffle(len(fn_monet))
testset = testset.map(parse_function, num_parallel_calls)
testset=testset.batch(1)
testset = testset.prefetch(num_parallel_calls)


_, ax = plt.subplots(5, 2, figsize=(20, 20))
for i, img in enumerate(testset.take(5)):
    prediction = photo_generator(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)
    ax[i, 0].imshow(img)
    ax[i, 1].imshow(prediction)
    ax[i, 0].set_title("Input Photo")
    ax[i, 1].set_title("Photo-esque")
    ax[i, 0].axis("off")
    ax[i, 1].axis("off")
plt.show()

In [ ]:
def evaluate_cycle(ds, generator_a, generator_b, n_samples=1):
    fig, axes = plt.subplots(n_samples, 3, figsize=(22, (n_samples*6)))
    axes = axes.flatten()
    
    ds_iter = iter(ds)
    for n_sample in range(n_samples):
        idx = n_sample*3
        example_sample = next(ds_iter)
        generated_a_sample = generator_a.predict(example_sample)
        generated_b_sample = generator_b.predict(generated_a_sample)
        
        axes[idx].set_title('Input image', fontsize=18)
        axes[idx].imshow(example_sample[0] * 0.5 + 0.5)
        axes[idx].axis('off')
        
        axes[idx+1].set_title('Generated image', fontsize=18)
        axes[idx+1].imshow(generated_a_sample[0] * 0.5 + 0.5)
        axes[idx+1].axis('off')
        
        axes[idx+2].set_title('Cycled image', fontsize=18)
        axes[idx+2].imshow(generated_b_sample[0] * 0.5 + 0.5)
        axes[idx+2].axis('off')
        
    plt.show()

In [ ]:
evaluate_cycle(testset.take(10), monet_generator, photo_generator, n_samples=10)

In [ ]:
import PIL

def predict_and_save(input_ds, generator_model, output_path):
    i = 1
    for img in input_ds:
        prediction = generator_model(img, training=False)[0].numpy() # make predition
        prediction = (prediction * 127.5 + 127.5).astype(np.uint8)   # re-scale
        im = PIL.Image.fromarray(prediction)
        im.save(output_path + str(i) + ".jpg")
        i += 1

In [ ]:
%%time
os.makedirs('./images1') # Create folder to save generated images
predict_and_save(photo_ds, monet_generator, './images1')

In [ ]:
shutil.make_archive('/kaggle/working/images1/', 'zip', './images1')#converting to zip
print(f"Generated samples: {len([name for name in os.listdir('./images1/') if os.path.isfile(os.path.join('./images1/', name))])}")